## **SBERT with Logistic Regression**
### **Davit Davtyan**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from sklearn.pipeline import Pipeline


from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pandas as pd
from datasets import load_dataset
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
ds = load_dataset("knkarthick/dialogsum")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


In [ ]:
clusters = pd.read_csv('dialogsum_clustered.csv')

train_df = pd.DataFrame(ds['train'])
validation_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

train_df = train_df.merge(clusters, on='id', how='left')
validation_df = validation_df.merge(clusters, on='id', how='left')
test_df = test_df.merge(clusters, on='id', how='left').dropna(subset=['cluster'])

train_df['combined_text'] = train_df['dialogue'] + " [SEP] " + train_df['summary']
validation_df['combined_text'] = validation_df['dialogue'] + " [SEP] " + validation_df['summary']
test_df['combined_text'] = test_df['dialogue'] + " [SEP] " + test_df['summary']

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from sklearn.pipeline import Pipeline


from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pandas as pd
from datasets import load_dataset
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


In [ ]:
# Example placeholders, replace or adjust with actual data preparation steps if needed
train_texts = train_df['combined_text'].tolist()
train_labels = train_df['cluster'].tolist()

validation_texts = validation_df['combined_text'].tolist()
validation_labels = validation_df['cluster'].tolist()

test_texts = test_df['combined_text'].tolist()
test_labels = test_df['cluster'].tolist()

In [ ]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
train_embeddings = sbert_model.encode(train_texts)
validation_embeddings = sbert_model.encode(validation_texts)
test_embeddings = sbert_model.encode(test_texts)

In [ ]:
classifier = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs'))
classifier.fit(train_embeddings, train_labels)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [ ]:
# Validation Evaluation
val_preds = classifier.predict(validation_embeddings)
val_accuracy = accuracy_score(validation_labels, val_preds)
val_f1 = f1_score(validation_labels, val_preds, average='weighted')
print(f'Validation Accuracy: {val_accuracy}, Validation F1 Score: {val_f1}')

# Test Evaluation
test_preds = classifier.predict(test_embeddings)
test_accuracy = accuracy_score(test_labels, test_preds)
test_f1 = f1_score(test_labels, test_preds, average='weighted')
print(f'Test Accuracy: {test_accuracy}, Test F1 Score: {test_f1}')

Validation Accuracy: 0.346, Validation F1 Score: 0.34777354679223504
Test Accuracy: 0.324, Test F1 Score: 0.3226528029419795
